In [126]:
import pandas as pd
import os
from pyDOE import *
from scipy.io import netcdf as nc
import xarray as xr
import copy

## Download latest version of params file from google drive
* requires 'publishing' the google drive spreadsheet
* file > publish to web
* then it can be set up to continuously publish the spreadsheet to a stable url (with some latency, maybe 1-2 minutes)
* note that the first tab must be the sheet where the relevant information is located

In [2]:
data_url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vQs413GtLXtHVDCqEPgAwn4BbDjoWmV7uFqOAWH4mgpxXoVfN6ijnJdhyRgLkV-n2eU-sSQush4CzYU/pub?output=csv'
#cmd = 'curl '+data_url+' > params.csv'
cmd = 'curl -L '+data_url+' > params.csv' # need to add -L option to force redirects
os.system(cmd)

0

## Defining a class for organizing parameter information

In [3]:
class ParamVal(object):
    """
    Stores parameter information
    """
    
    def __init__(self, name, loc, minval=None, maxval=None, defval=None):
        self._name = name # parameter name
        self._min = minval # minimum value
        self._max = maxval # maximum value
        self._default = defval # default value
        self._value = None # actual value to be used in a given ensemble member
        self._location = loc # location of parameter (params file or namelist)

    @property
    def name(self):
        return self._name

    @property
    def min(self):
        return self._min

    @property
    def max(self):
        return self._max
    
    @property
    def default(self):
        return self._default
    
    @property
    def value(self):
        return self._value
    
    @property
    def location(self):
        return self._location
    
    @name.setter
    def name(self, new_name):
        self._name = new_name
   
    @min.setter
    def min(self, new_min):
        self._min = new_min
        
    @max.setter
    def max(self, new_max):
        self._max = new_max
        
    @default.setter
    def default(self, new_def):
        self._default = new_def
        
    @value.setter
    def value(self, new_val):
        self._value = new_val
        
    #def calc_value(self, sampling_protocol):
    #    """
    #    Calculates parameter value
    #    """
    #    
    #    if sampling_protocol == "OAAT":
    #            self._value = self._default # as an example
    #    elif sampling_protocol == "LHC":
    #           pass
            
    def __repr__(self):
        return "%s:\n\tloc = %s\n\tdefault = %s\n\tmin = %s\n\tmax = %s\n\tvalue = %s" % (self.name, self.location, self.default, self.min, self.max, self.value)

In [4]:
# TO DO: add these unit tests to ParamVal class itself
# Check every time the code updates, or adding new functionality

# testing out the class/dictionary functionality
test_dict = {"P1": ParamVal("P1", minval=0.0, maxval=1.0, defval=2.0, loc='N'),
             "P2": ParamVal("P2", minval=[0,0,0,0,0], maxval=[100,100,100,100,100], defval=[0,1,2,3,4], loc='P'),
             "P3": ParamVal("P3", minval="min", maxval="max", defval="value", loc='N'),
             "P4": ParamVal("P4", loc='P')
            }

# example of adding a new parameter
test_dict["new_param"] = ParamVal("new_param", 'N')

# example of setting the max value
test_dict["P4"].max = 200

# look at the test dictionary
for key in test_dict:
    print(test_dict[key])

P1:
	loc = N
	default = 2.0
	min = 0.0
	max = 1.0
	value = None
P2:
	loc = P
	default = [0, 1, 2, 3, 4]
	min = [0, 0, 0, 0, 0]
	max = [100, 100, 100, 100, 100]
	value = None
P3:
	loc = N
	default = value
	min = min
	max = max
	value = None
P4:
	loc = P
	default = None
	min = None
	max = 200
	value = None
new_param:
	loc = N
	default = None
	min = None
	max = None
	value = None


## Defining a class for organizing ensemble members

In [135]:
class Member(object):
    """
    Stores and works with a bunch of ParamVals
    """

    def __init__(self, name, paramvals):
        self._name = name
        self._paramvals = paramvals
        
    @property
    def name(self):
        return self._name
    
    @property
    def paramvals(self):
        return self._paramvals
    
    @name.setter
    def name(self, new_name):
        self._name = new_name
    
    #def calc_param_vals(self, sampling_protocol):
    #    """
    #    Loops over paramvals and calls calc_value(sampling_protocol)
    #    """
    #    
    #    for paramvals in self._paramvals:
    #        self._paramvals[paramvals].calc_value(sampling_protocol)
    
    def get_names(self):
        """
        Returns a list of parameter names.
        """
        names = []
        for param in self._paramvals:
            names.append(self._paramvals[param].name)
        return names
                 
    def write(self, sampling_protocol):
        pass
        
    def __repr__(self):
        return "I am a member named %s" % (self._name, )

In [136]:
# TO DO: add these unit tests to Member class itself
# Check every time the code updates, or adding new functionality

# testing out the class/dictionary functionality
member_test_dict = {"M1": Member("M1", paramvals=test_dict),
             "M2": Member("M2", paramvals=test_dict),
             "M3": Member("M3", paramvals=None),
             "M4": Member("M4", paramvals=None)
            }

# example of adding a new member
member_test_dict["new_member"] = Member("new_member", paramvals=test_dict)

# example of setting the name
member_test_dict["new_member"].name = "M5"

# look at the test dictionary
for key in member_test_dict:
    print(member_test_dict[key])

I am a member named M1
I am a member named M2
I am a member named M3
I am a member named M4
I am a member named M5


In [137]:
# look at a member's paramvals in the test dictionary
member_test_dict['M1'].paramvals
# look at a member's specific parameter
member_test_dict['M1'].paramvals['P1']
# look at the list of parameter names
member_test_dict['M1'].get_names()

['P1', 'P2', 'P3', 'P4', 'new_param']

## Define a class for organizing the ensemble

In [171]:
class Ensemble(object):
    """
    Stores and works with a bunch of Members
    """
    
    # assign the basepftfile
    _basepftfile = "../basecase/clm5_params.c200519.nc"
    
    # NOTE: here using an example lnd_in file to pull in default namelist values
    # Could also parse the namelist defaults file, see: https://github.com/ESCOMP/CTSM/blob/e2b9745d81ed5cb7cd7f5d6098edf506a4956335/bld/namelist_files/namelist_defaults_ctsm.xml
    _thedir = '/glade/work/djk2120/ctsm_hardcode_co/cime/scripts/clm50c6_ctsmhardcodep_2deg_GSWP3V1_Sparse250_2000/CaseDocs/'
    _thefil = 'lnd_in'
    _lndin = _thedir+_thefil
    
    _params_dict = {}
    
    def __init__(self, csvfile, sampling_protocol):   
        # Read in csv data, filtering by the "include" column
        #data     = pd.read_csv('params.csv')
        data = pd.read_csv(csvfile,header=0,skiprows=[1]) # modify read_csv to account for header spanning 2 rows
        included = data['include'] == 1
        params_full = data.loc[included,['name','location','min','max','pft_mins','pft_maxs']]

        # reset indexing and get rid of excel row number
        params = params_full.reset_index(drop=True)
        
        # get number of parameters
        self._nparam = len(params['name'])
        
        # read in default file
        def_params = xr.open_dataset(self._basepftfile)

        # declare a dictionary to store parameter information
        #self._params_dict={}
        
        # reading in the default values
        # loop over parameters grabbing name and location
        for name,loc in zip(params['name'],params['location']):      
            
            # select parameters located in the params file only
            if loc=='P':
                # getting parameter dims (i.e., checking for segment variation)
                dims = len(def_params[name].values.shape)
                if dims<2:
                    # no segment variation
                    x = def_params[name].values
                else:
                    # segment variation: kmax,ck,psi50,rootprof_beta
                    # assumes same values applied across segments
                    # TO DO: check this assumption, appears not true for rootprof_beta
                    x = def_params[name][0,:].values
                self._params_dict[name] = ParamVal(name, defval=x, loc='P')
            
            # select namelist parameters
            elif loc=='N':
                # build a command to search lndin for the parameter by name and put output in a tmp file
                cmd = 'grep '+name+' '+self._lndin+' > tmp.txt'
                ret = os.system(cmd)
                # checking for nonzero return code (exit status?), meaning parameter is not found
                if ret != 0:
                    # TO DO: will need to address these special cases somehow...
                    print(name+' not found')
                else:
                    f = open('tmp.txt', 'r')
                    # parse the value from the parameter name
                    tmp = f.read().split()[2]
                    f.close()
                    # cases where scientific notation(?) is specified by a "d"
                    # TO DO: there may be other special cases as well (scientific notation as an "e"?)
                    if 'd' in tmp:
                        tmp = tmp.split('d')
                        x = float(tmp[0])*10**float(tmp[1])
                    else:
                        x = float(tmp)
                    self._params_dict[name] = ParamVal(name, defval=x, loc='N')
        
        # assigning min and max values
        if sampling_protocol == 'OAAT':
            for i in range(self._nparam):
                # check for pft variation
                if params['min'].values[i]=='pft':
                    self._params_dict[params['name'].values[i]].min = np.fromstring(params['pft_mins'][i],dtype='float',sep=',')
                    self._params_dict[params['name'].values[i]].max = np.fromstring(params['pft_maxs'][i],dtype='float',sep=',')
                # check for "XXpercent" perturb from default
                elif "percent" in params['min'].values[i]:
                    percent_perturb = float(params['min'].values[i].split("percent")[0])
                    percent_min_values = self._params_dict[params['name'].values[i]].default*(1 - percent_perturb/100)
                    percent_max_values = self._params_dict[params['name'].values[i]].default*(1 + percent_perturb/100)            
                    self._params_dict[params['name'].values[i]].min = percent_min_values
                    self._params_dict[params['name'].values[i]].max = percent_max_values        
                else:
                    # assign min/max values directly
                    self._params_dict[params['name'].values[i]].min = params['min'].values[i]
                    self._params_dict[params['name'].values[i]].max = params['max'].values[i]
        elif sampling_protocol == 'LHC':
            # TO DO: assign LHC min/maxes
            pass
        
        # declare a dictionary to store member information
        self._members = {}
        
        # set the number of ensemble members
        if sampling_protocol == 'OAAT':
            # number of samples is twice the number of parameters (min and max perturbations)
            nsamp = 2*self._nparam
        elif sampling_protocol == 'LHC':
            # define sample size for LHC (user-specified)
            nsamp = 10
        
        # create the members
        # need to "deepcopy" the dictionary for each member so they can be modified independently
        for i in range(nsamp):
            self._members[i] = Member(str(i), copy.deepcopy(self._params_dict))
        
        # loop over members and calculate parameter values for each member
        # TO DO: THIS LOGIC IS BROKEN
        
        # DEBUGGING
        #doneparams = []
        #for paramname in self._members[0].get_names():
            #print(paramname)
            #for member in self._members:
                #print(member)
                #print("before: %s" % self._members[member].paramvals[paramname].value)
                #print(self._members[member].paramvals)
                #self._members[member].paramvals[paramname].value = "hello"
                #print("after: %s" % self._members[member].paramvals[paramname].value)
            #continue
            
        doneparams = []
        for member in self._members:
            #print(member)
            for paramname in self._members[member].get_names():
                # check if this parameter has already been assigned
                if paramname in doneparams:
                    continue
                    #break
                # check if this is a min or max perturbation
                if int(self._members[member].name)%2 == 0:
                    # min values
                    #print('value before')
                    #print(self._members[member].paramvals[paramname].value)
                    self._members[member].paramvals[paramname].value = self._members[member].paramvals[paramname].min
                    #print('value after')
                    #print(self._members[member].paramvals[paramname].value)
                    break
                else:
                    # max values
                    #print('value before')
                    #print(self._members[member].paramvals[paramname].value)
                    self._members[member].paramvals[paramname].value = self._members[member].paramvals[paramname].max
                    #print('value after')
                    #print(self._members[member].paramvals[paramname].value)
                    # parameter is "done" after creating min/max ensemble members
                    doneparams.append(paramname)
                    break

        
    @property
    def params(self):
        return self._params_dict
    
    @property
    def members(self):
        return self._members
    
    def output_files(self):
        for member in self._ens_members:
            member.save_psets()

In [172]:
# example of how to use this Ensemble class
newEns = Ensemble('params.csv', 'OAAT')
# get the params_dict - but which member do the values correspond to?
#newEns.params

# example of how to print member info
#for member in newEns.members:
    #print(newEns.members[member])
    #print(newEns.members[member].paramvals)
    
# example of how to print paramvals for a given member
#newEns.members[0].paramvals

# example of how to print vals of a specific parameter for a given member
#newEns.members[0].paramvals['dleaf']

# example of how to print all the vals of a specific parameter for all members
#for member in newEns.members:
#    print(newEns.members[member])
#    print(newEns.members[member].paramvals['dleaf'])

# example of how to print all the "value" vals of a specific parameter for all members
for member in newEns.members:
    print(newEns.members[member].paramvals['baseflow_scalar'].value)

# debugging...
#newEns.members[0].get_names()
#int(newEns.members[2].name)%2==0
#newEns.members[0].paramvals['dleaf'].value

None
None
0.0005
0.1
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


## Generate parameter sampling
 * ### careful, each time you run LHC you get a new random draw

In [ ]:
if sampling_protocol == 'LHC':
    # define sample size (number of ensemble members)
    nsamp = 10

    # Generate the latin hypercube sample
    lhd = lhs(nparam, samples=int(nsamp))
    # lhd is a 2D array indexed by ensemble member x parameter
    
    # figure out how many pft-dependent params there are in this sample
    npftparam = sum(params['min']=='pft')
    
    if npftparam>0:
        # get dataframe index of first pft param
        pftfirstind = params.index[params['min']=='pft'][0]
        
        # get number of pfts
        npft = len(np.fromstring(params['pft_mins'][pftfirstind],dtype='float',sep=','))
        
        # set up numpy array to store pft-specific values
        pft_array = np.nan*np.ones([npftparam,npft,nsamp])
        
        for j in range(npftparam):
            # get the index for the current pft param
            pftind = params.index[params['min']=='pft'][j]
            
            # get min values
            min_pft_array = np.fromstring(params['pft_mins'][pftind],dtype='float',sep=',')
            # max values
            max_pft_array = np.fromstring(params['pft_maxs'][pftind],dtype='float',sep=',')
            
            # loop over samples and calculate parameter values for each pft
            for i in range(nsamp):
                pft_array[j,:,i] = (max_pft_array - min_pft_array)*lhd[i,pftind] + min_pft_array
                # can't store pft_array as a pandas dataframe because it's 3D
                # unless there is some alternate way to store this data?
    
    # initialize min/max arrays - for params without pft-variation
    min_array = np.nan*np.ones(nparam)
    max_array = np.nan*np.ones(nparam)
    
    # generate arrays with min and max values
    for i in range(nparam):
        if params['min'].values[i]=='pft':
            # TO DO: what's a good placeholder, to denote need to reference pft_array?
            # numpy doesn't like assigning a string to an existing array of floats
            # for now, just print a message
            print('skipping '+params['name'].values[i]+'...this parameter varies with PFT')
            
            # Numpy doesn't like assigning an array to a single index in an existing array
            # The problem is still that I'm declaring min_array before trying to assign values
            # If I could build it all at once, numpy would allow for nested arrays
            #min_array[i] = np.fromstring(params['pft_mins'].values[i],dtype='float',sep=',')
            #max_array[i] = np.fromstring(params['pft_maxs'].values[i],dtype='float',sep=',')
        else:
            # assign min/max values
            min_array[i] = float(params['min'].values[i])
            max_array[i] = float(params['max'].values[i])
            
    # calculate parameter values; skip pft params (NaNs in min/max arrays)
    param_array = (max_array - min_array)*lhd + min_array

elif sampling_protocol == 'OAAT':
    # number of samples is twice the number of parameters (min and max perturbations)
    #nsamp = 2*nparam
    
    # set up parameter array
    # NaN is code for keep the default value
    #param_array = np.nan*np.ones([nsamp,nparam])
    
    # get the min and max indices (even/odd rows)
    #mins_index = (np.arange(0,nsamp,2),np.arange(0,nparam,1))
    #maxs_index = (np.arange(1,nsamp,2),np.arange(0,nparam,1))
    
    # figure out how many pft-dependent params there are in this sample
    #npftparam = sum(params['min']=='pft')
    
    # set up numpy array to store pft-specific values
    #if npftparam>0:
        # get dataframe index of first pft param
        #pftfirstind = params.index[params['min']=='pft'][0]
        
        # get number of pfts
        #npft = len(np.fromstring(params['pft_mins'][pftfirstind],dtype='float',sep=','))
        
        # third dimension accounts for min/max values
        #pft_array = np.nan*np.ones([npftparam,npft,2])
        
        #for j in range(npftparam):
            # get the index for the current pft param
            #pftind = params.index[params['min']=='pft'][j]
            
            # assign the values for min and max
            #pft_array[j,:,0]=np.fromstring(params['pft_mins'][pftind],dtype='float',sep=',')
            #pft_array[j,:,1]=np.fromstring(params['pft_maxs'][pftind],dtype='float',sep=',')
            # can't store pft_array as a pandas dataframe because it's 3D
            # unless there is some alternate way to store this data?
        
    # assign values to the parameter array
    for i in range(nparam):
        # check for pft variation
        if params['min'].values[i]=='pft':
            # TO DO: what's a good placeholder, to denote need to reference pft_array?
            # e.g., param_array[mins_index[0][i]][i] = float('pft')
            # but numpy doesn't like assigning a string to an existing array of floats
            # for now, just print a message
            #print('skipping '+params['name'].values[i]+'...this parameter varies with PFT')
            params_dict[params['name'].values[i]].min = np.fromstring(params['pft_mins'][i],dtype='float',sep=',')
            params_dict[params['name'].values[i]].max = np.fromstring(params['pft_maxs'][i],dtype='float',sep=',')
        # check for "XXpercent" perturb from default
        elif "percent" in params['min'].values[i]:
            #print('skipping '+params['name'].values[i]+'...for now, need default values')
            percent_perturb = float(params['min'].values[i].split("percent")[0])
            percent_min_values = params_dict[params['name'].values[i]].default*(1 - percent_perturb/100)
            percent_max_values = params_dict[params['name'].values[i]].default*(1 + percent_perturb/100)            
            params_dict[params['name'].values[i]].min = percent_min_values
            params_dict[params['name'].values[i]].max = percent_max_values        
        else:
            # assign min/max values directly
            #param_array[mins_index[0][i]][i]=params['min'].values[i]
            #param_array[maxs_index[0][i]][i]=params['max'].values[i]
            params_dict[params['name'].values[i]].min = params['min'].values[i]
            params_dict[params['name'].values[i]].max = params['max'].values[i]

# store psets in a pandas dataframe
#psets = pd.DataFrame(data=param_array, index=None, columns=params['name'])
#psets

params_dict

### Some notes on the 12-parameter test ensemble:
* FUN_fracfixers has single values for min/max specified in spreadsheet, but these values needs to be applied across all PFTs 
* leafcn has a non-vegetated default value of 1, so the min/max arrays change this value when a percent perturbation is applied
* don't currently have checks to see if min or max are equivalent to default value (don't change params files / don't make a namelist mod?) - precision/round-off errors could be important here

### Modify psets dataframe to include pft flag

In [64]:
#if sampling_protocol == 'LHC':
#    for ind,name in enumerate(params['name']):
#        # check for NaNs in the whole column (denotes PFT-specific param)
#        if np.isnan(psets[name]).all():
#            print('adding pft flag for '+name)
#            psets[name] = 'pft'

# NOTE: this bit of code generates a pandas warning, but still executes as it should
# Could come back to this if we figure out how to put some pft flag in the preceding code
#elif sampling_protocol == 'OAAT':    
#    for ind,name in enumerate(params['name']):
        # check for NaNs in the whole column (denotes PFT-specific param)
#        if np.isnan(psets[name]).all():
#            print('adding pft flag for '+name)
#            psets[name][mins_index[0][ind]] = 'pft'
#            psets[name][maxs_index[0][ind]] = 'pft'

#psets

### Check out pft_array, the numpy array that stores pft-specific values

In [65]:
#pft_array.shape 
# OAAT dims are (npftparam, npft, 2) where last dim represents min/max perturbations
# LHC dims are (npftparam, npft, nsamp)

## Generate parameter files
* ### this will overwrite parameter files!!
* ### proceed with caution

In [52]:
# assign the basepftfile
basepftfile = "../basecase/clm5_params.c200519.nc"

if sampling_protocol == 'OAAT':
    # initialize npftparam counter
    #npftparam = 0
    # number of samples is twice the number of parameters (min and max perturbations)
    nsamp = 2*nparam

# loop over nsamp and modify the parameter values accordingly
for i in range(nsamp):
    if sampling_protocol == 'OAAT':
        # open the default file (twice for OAAT)
        tmp_min = xr.open_dataset(basepftfile)
        tmp_max = xr.open_dataset(basepftfile)
                
        # generate name for this param file
        min_pftfile = "../paramfiles/"+prefix+str(2*i+1).zfill(4)+".nc"
        max_pftfile = "../paramfiles"+prefix+str(2*i+2).zfill(4)+".nc"
        print('working on '+min_pftfile+' and '+max_pftfile)
        
    # open the default file
    #tmp = xr.open_dataset(basepftfile)
    
    # generate name for this param file
    #pftfile = "../paramfiles/"+prefix+str(i+1).zfill(4)+".nc"
    #print('working on '+pftfile)
    
    if sampling_protocol == 'LHC':
        # reset npftparam counter for each sample
        npftparam = 0
    
    # loop over parameters
    for name,loc in zip(params['name'],params['location']):
        
        # select parameters located in the params file only
        if loc=='P':

            if sampling_protocol == 'LHC':
                print(name+' modified')
                var = tmp[name]
                
                # check to see if there is pft variation
                if psets[name][i]=='pft':
                    
                    # check which npftparam we are on
                    print('npftparam='+str(npftparam))
                    
                    # modify values
                    tmp[name][:] = pft_array[npftparam,:,i]
                    
                    # increment npftparam counter; only do this once per parameter
                    npftparam += 1
                    
                else: # no pft variation, assign the same number across all PFTs (as applicable)
                    
                    # check for indexing by pft
                    # NOTE: this logic might get tripped up by froz_q10 and q10_mr which are currently indexed by a placeholder dim "allpfts" (should be removed soon)
                    if var.shape:
                        
                        # check for indexing by segment or variants, which will be the first dimension
                        # skip the first index, don't want to overwrite non-vegetated values
                        if var.shape[0] != npft: 
                            tmp[name][:,1:] = psets[name][i]
                        else: # indexed by pft only
                            tmp[name][1:] = psets[name][i]
                        
                    else: # single value, no indexing by pft
                        tmp[name] = psets[name][i]
            
            elif sampling_protocol == 'OAAT':          
                # check to see if this parameter should be modified
                # logic is checking for psets that are NOT NaNs
                #if pd.isna(psets[name][i])==False:
                print(name+' modified')
                var = tmp[name]
                    #print(var.shape)

                    # check to see if there is pft variation
                    # NOTE: may want to use only first 16 indices for this ensemble (no crop), in which case indexing changes 
                    #if psets[name][i]=='pft':
                    
                        # check which npftparam we are on
                        #print('npftparam='+str(npftparam))
                    
                        # check if this is a min or max perturbation
                        #if i%2==0:
                            #tmp[name][:] = pft_array[npftparam,:,0] # min values
                        #else:
                            #tmp[name][:] = pft_array[npftparam,:,1] # max values

                            # increment npftparam counter; only do this once per parameter
                            #npftparam += 1 
                
                    #else: # no pft variation, assign the same number across all PFTs (as applicable)
                    
                # check for indexing by pft
                # NOTE: this logic might get tripped up by froz_q10 and q10_mr which are currently indexed by a placeholder dim "allpfts" (should be removed soon)
                if var.shape:
                        
                    # check for indexing by segment or variants, which will be the first dimension
                    # skip the first index, don't want to overwrite non-vegetated values
                    if var.shape[0] != npft: 
                        #tmp[name][:,1:] = psets[name][i]
                        tmp_min[name][:,1:] = params_dict[name].min
                    else: # indexed by pft only
                        #tmp[name][1:] = psets[name][i]
                        tmp_min[name][1:] = params_dict[name].min
                    
                else: # single value, no indexing by pft
                    #tmp[name] = psets[name][i]
                    tmp_min[name] = params_dict[name].min

    # write changes (if any) to file
    tmp_min.to_netcdf(pftfile,'w')

working on ../paramfiles/OAAT0001.nc
displar modified
working on ../paramfiles/OAAT0002.nc
displar modified
working on ../paramfiles/OAAT0003.nc
dleaf modified
npftparam=0
working on ../paramfiles/OAAT0004.nc
dleaf modified
npftparam=0
working on ../paramfiles/OAAT0005.nc
working on ../paramfiles/OAAT0006.nc
working on ../paramfiles/OAAT0007.nc
working on ../paramfiles/OAAT0008.nc
working on ../paramfiles/OAAT0009.nc
fff modified
working on ../paramfiles/OAAT0010.nc
fff modified
working on ../paramfiles/OAAT0011.nc
medlynslope modified
npftparam=1
working on ../paramfiles/OAAT0012.nc
medlynslope modified
npftparam=1
working on ../paramfiles/OAAT0013.nc
kmax modified
npftparam=2
working on ../paramfiles/OAAT0014.nc
kmax modified
npftparam=2


## Generate namelist files

Bash script will generate the namelist mod for pointing to the right params file

In [53]:
# create the namelist mod files
for i in range(nsamp):
    nlfile = "../namelist_mods/"+prefix+str(i+1).zfill(4)+".txt" 
    with open(nlfile,"w") as file:
        output = "! user_nl_clm namelist options written by generate_params:\n"
        file.write(output)

# populate with mods
for name,loc in zip(params['name'],params['location']):
    if loc=='N':
        # don't have to worry about pft-variation here because namelist params won't have that
        for i in range(nsamp):
            
            if sampling_protocol == 'LHC':
                nlfile = "../namelist_mods/"+prefix+str(i+1).zfill(4)+".txt"
                print('working on '+nlfile)
                with open(nlfile,"a") as file: # key is using "a" for append option
                    print(name+' modified')
                    output = "%s=%s\n" % (name, psets[name][i]) #round??
                    file.write(output) 
            
            elif sampling_protocol == 'OAAT': 
                # check to see if this parameter should be modified
                # logic is checking for psets that are NOT NaNs
                if ~np.isnan(psets[name][i]):
                    nlfile = "../namelist_mods/"+prefix+str(i+1).zfill(4)+".txt"
                    print('working on '+nlfile)
                    with open(nlfile,"a") as file: # key is using "a" for append option
                        print(name+' modified')
                        output = "%s=%s\n" % (name, psets[name][i]) #round??
                        file.write(output) 

working on ../namelist_mods/OAAT0005.txt
baseflow_scalar modified
working on ../namelist_mods/OAAT0006.txt
baseflow_scalar modified
working on ../namelist_mods/OAAT0007.txt
maximum_leaf_wetted_fraction modified
working on ../namelist_mods/OAAT0008.txt
maximum_leaf_wetted_fraction modified


## Save off the parameter sets

In [54]:
# create a name for this particular ensemble
ensemble_name = "test0001"
# build the file name with the prefix (ensemble type)
psetsfile = "../parameter_sets/"+prefix+"_"+ensemble_name+".csv"
#print(psetsfile)

# first, save the psets dataframe to csv
psets.to_csv(psetsfile)

# second, save the pft array (if applicable)
pftarrayfile = "../parameter_sets/"+prefix+"_"+ensemble_name+"_pftvals"
#print(pftarrayfile)
# save as a numpy array (for now, easiest solution for 3D array?)
np.save(pftarrayfile, pft_array)
# example of how to load it back in
#test = np.load(pftarrayfile+".npy")